# Load Data and Merge into Proper DF to Save as CSV file

In [1]:
import os
os.chdir('../saliency_model/')

import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib

import pandas as pd
import seaborn as sns
from glob import glob

matplotlib.rcParams.update({'font.size': 15})

### Pick Metric
NSS, SIM, IG, AUC

In [2]:
plot_metric = 'AUC'

In [3]:
# pick directory
save_dir ='../report/plots/'

In [4]:
os.chdir('../results/')
glob('*.p')

['19-02-01_results_over_baseline_faces_07.p',
 '19-02-01_results_plain_models.p',
 '19-02-01_results_face_07.p',
 '2019-05-12_results-with-std.p',
 '19-02-01_results_over_baseline_faces_05.p',
 '2019-05-11_results-with-std.p',
 '19-02-01_results_over_baseline_faces_03.p',
 '19-01-31_results.p',
 '19-02-01_results_face_03.p',
 '19-02-01_results_face_05.p']

## Pick the Dataset and Load Required Data

In [5]:
(summary, context, model, metric) = pickle.load(open( '19-02-01_results_plain_models.p', "rb"))

In [6]:
print(np.shape(summary))
print(context)
print(model)
print(metric)

(6, 9, 4)
['all', 'person', 'no person', 'animal', 'electronic', 'sports']
['aim', 'cas', 'covsal', 'gbvs', 'icf', 'ik', 'sun', 'dg', 'faces']
['nss', 'sim', 'ig', 'auc']


In [7]:
(summary_f03, context_f03, model_f03, metric_f03) = pickle.load(open( '19-02-01_results_face_03.p', "rb"))
(summary_f05, context_f05, model_f05, metric_f05) = pickle.load(open( '19-02-01_results_face_05.p', "rb"))
(summary_f07, context_f07, model_f07, metric_f07) = pickle.load(open( '19-02-01_results_face_07.p', "rb"))

#### Vizualize the inside of one of the models

In [8]:
model_f03

['aim_face_03',
 'cas_face_03',
 'covsal_face_03',
 'gbvs_face_03',
 'icf_face_03',
 'ik_face_03',
 'sun_face_03']

## Data Loading

### Get ID of the Metric

In [9]:
if plot_metric == 'NSS':
    metric_id = 0
elif plot_metric == 'SIM':
    metric_id = 1
elif plot_metric == 'IG':
    metric_id = 2
elif plot_metric == 'AUC':
    metric_id = 3
else:
    raise ValueError('Wrong metric name provided.')
    
metric_id

3

### Load the scores

In [10]:
scores = summary[:,:,metric_id]
f03_scores = summary_f03[:,:,metric_id]
f05_scores = summary_f05[:,:,metric_id]
f07_scores = summary_f07[:,:,metric_id]

## Change scores into DataFrames
### No Faces Data Frame

In [11]:
df = pd.DataFrame(data=scores, columns=model, index=context)
df.columns.name='models'
df.index.name='context'

df = df.transpose()
df

context,all,person,no person,animal,electronic,sports
models,,,,,,
aim,0.698040,0.707347,0.682952,0.681585,0.683133,0.729921
cas,0.692052,0.703851,0.672925,0.667086,0.670638,0.728063
covsal,0.746774,0.754006,0.735052,0.733442,0.726257,0.767142
gbvs,0.749465,0.758711,0.734478,0.734344,0.728549,0.783232
icf,0.701568,0.711141,0.686049,0.679900,0.682979,0.717650
ik,0.663008,0.673263,0.646383,0.656406,0.652015,0.688133
sun,0.603264,0.599431,0.609479,0.560426,0.589477,0.595790
dg,0.760010,0.773875,0.737536,0.749271,0.738996,0.784495
faces,0.979706,0.968213,0.998336,0.994076,0.968691,0.985565


#### Separate Faces From the rest of the results

In [12]:
faces = df.loc['faces']
df = df.loc[['aim','cas', 'covsal', 'gbvs', 'icf', 'ik', 'sun', 'dg']]

#### Get Mean Values across Contexts and Models

In [13]:
df['mean context'] = df.mean(axis=1)
df = df.transpose()
df['mean model'] = df.mean(axis=1)
df = df.transpose()
df

context,all,person,no person,animal,electronic,sports,mean context
models,,,,,,,
aim,0.698040,0.707347,0.682952,0.681585,0.683133,0.729921,0.697163
cas,0.692052,0.703851,0.672925,0.667086,0.670638,0.728063,0.689103
covsal,0.746774,0.754006,0.735052,0.733442,0.726257,0.767142,0.743779
gbvs,0.749465,0.758711,0.734478,0.734344,0.728549,0.783232,0.748130
icf,0.701568,0.711141,0.686049,0.679900,0.682979,0.717650,0.696548
ik,0.663008,0.673263,0.646383,0.656406,0.652015,0.688133,0.663201
sun,0.603264,0.599431,0.609479,0.560426,0.589477,0.595790,0.592978
dg,0.760010,0.773875,0.737536,0.749271,0.738996,0.784495,0.757364
mean model,0.701773,0.710203,0.688107,0.682807,0.684005,0.724303,0.698533


In [14]:
df_mean_model = df.loc['mean model']
df_mean_context = df.transpose().loc['mean context']

### Faces with $\alpha$ = 0.3

In [15]:
df_f03 = pd.DataFrame(data=f03_scores, columns=model_f03, index=context_f03)
df_f03.columns.name='models'
df_f03.index.name='context'

# Rename columns of the face plot
col = {'aim_face_03':'aim', 'cas_face_03':'cas', 'covsal_face_03':'covsal', 'gbvs_face_03':'gbvs',
       'icf_face_03':'icf', 'ik_10_face_03':'ik_10', 'sun_face_03':'sun', 'ik_face_03':'ik'}
df_f03 = df_f03.rename(index=str, columns=col).transpose()

df_f03

context,all,person,no person,animal,electronic,sports
models,,,,,,
aim,0.701392,0.713620,0.681570,0.681863,0.689665,0.730011
cas,0.695769,0.710621,0.671564,0.668491,0.676538,0.729025
covsal,0.750548,0.760145,0.734990,0.735682,0.732640,0.768929
gbvs,0.752009,0.763010,0.734177,0.735972,0.732871,0.784090
icf,0.703955,0.715215,0.685701,0.680453,0.686559,0.719037
ik,0.668251,0.682089,0.645819,0.658155,0.661111,0.690817
sun,0.611347,0.613150,0.608425,0.562480,0.601873,0.599777


#### Get Mean Values across Contexts and Models

In [16]:
df_f03['mean context'] = df_f03.mean(axis=1)
df_f03 = df_f03.transpose()
df_f03['mean model'] = df_f03.mean(axis=1)
df_f03 = df_f03.transpose()
df_f03

context,all,person,no person,animal,electronic,sports,mean context
models,,,,,,,
aim,0.701392,0.713620,0.681570,0.681863,0.689665,0.730011,0.699687
cas,0.695769,0.710621,0.671564,0.668491,0.676538,0.729025,0.692001
covsal,0.750548,0.760145,0.734990,0.735682,0.732640,0.768929,0.747156
gbvs,0.752009,0.763010,0.734177,0.735972,0.732871,0.784090,0.750355
icf,0.703955,0.715215,0.685701,0.680453,0.686559,0.719037,0.698487
ik,0.668251,0.682089,0.645819,0.658155,0.661111,0.690817,0.667707
sun,0.611347,0.613150,0.608425,0.562480,0.601873,0.599777,0.599509
mean model,0.697610,0.708264,0.680321,0.674728,0.683037,0.717384,0.693557


In [17]:
df_f03_mean_model = df_f03.loc['mean model']
df_f03_mean_context = df_f03.transpose().loc['mean context']

### Faces with $\alpha$ = 0.5

In [18]:
df_f05 = pd.DataFrame(data=f05_scores, columns=model_f05, index=context_f05)
df_f05.columns.name='models'
df_f05.index.name='context'

# Rename columns of the face plot
col = {'aim_face_05':'aim', 'cas_face_05':'cas', 'covsal_face_05':'covsal', 'gbvs_face_05':'gbvs',
       'icf_face_05':'icf', 'ik_10_face_05':'ik_10', 'sun_face_05':'sun', 'ik_face_05':'ik'}
df_f05 = df_f05.rename(index=str, columns=col).transpose()

df_f05

context,all,person,no person,animal,electronic,sports
models,,,,,,
aim,0.702716,0.715734,0.681613,0.682210,0.692044,0.730439
cas,0.697598,0.713322,0.672109,0.668932,0.680393,0.729163
covsal,0.751736,0.762012,0.735077,0.736452,0.734824,0.769649
gbvs,0.752898,0.764442,0.734184,0.736536,0.734617,0.784087
icf,0.704762,0.716543,0.685663,0.680618,0.687945,0.719901
ik,0.669794,0.684588,0.645813,0.658868,0.664084,0.691309
sun,0.613976,0.617384,0.608452,0.563758,0.605716,0.601674


#### Get Mean Values across Contexts and Models

In [19]:
df_f05['mean context'] = df_f05.mean(axis=1)
df_f05 = df_f05.transpose()
df_f05['mean model'] = df_f05.mean(axis=1)
df_f05 = df_f05.transpose()
df_f05

context,all,person,no person,animal,electronic,sports,mean context
models,,,,,,,
aim,0.702716,0.715734,0.681613,0.682210,0.692044,0.730439,0.700793
cas,0.697598,0.713322,0.672109,0.668932,0.680393,0.729163,0.693586
covsal,0.751736,0.762012,0.735077,0.736452,0.734824,0.769649,0.748292
gbvs,0.752898,0.764442,0.734184,0.736536,0.734617,0.784087,0.751127
icf,0.704762,0.716543,0.685663,0.680618,0.687945,0.719901,0.699239
ik,0.669794,0.684588,0.645813,0.658868,0.664084,0.691309,0.669076
sun,0.613976,0.617384,0.608452,0.563758,0.605716,0.601674,0.601827
mean model,0.699069,0.710575,0.680416,0.675339,0.685660,0.718032,0.694848


In [20]:
df_f05_mean_model = df_f05.loc['mean model']
df_f05_mean_context = df_f05.transpose().loc['mean context']

### Faces with $\alpha$ = 0.7

In [21]:
df_f07 = pd.DataFrame(data=f07_scores, columns=model_f07, index=context_f07)
df_f07.columns.name='models'
df_f07.index.name='context'

# Rename columns of the face plot
col = {'aim_face_07':'aim', 'cas_face_07':'cas', 'covsal_face_07':'covsal', 'gbvs_face_07':'gbvs',
       'icf_face_07':'icf', 'ik_10_face_07':'ik_10', 'sun_face_07':'sun', 'ik_face_07':'ik'}
df_f07 = df_f07.rename(index=str, columns=col).transpose()

df_f07

context,all,person,no person,animal,electronic,sports
models,,,,,,
aim,0.703589,0.717085,0.681711,0.682661,0.693656,0.731142
cas,0.698546,0.714773,0.672241,0.669232,0.682526,0.729582
covsal,0.753049,0.763897,0.735464,0.737088,0.737128,0.770830
gbvs,0.753201,0.764895,0.734245,0.736809,0.735714,0.784198
icf,0.705782,0.718012,0.685956,0.681121,0.690002,0.721120
ik,0.670892,0.686282,0.645943,0.659359,0.666581,0.692061
sun,0.615479,0.619786,0.608496,0.564438,0.608479,0.602998


In [22]:
df_f07['mean context'] = df_f07.mean(axis=1)
df_f07 = df_f07.transpose()
df_f07['mean model'] = df_f07.mean(axis=1)
df_f07 = df_f07.transpose()
df_f07

context,all,person,no person,animal,electronic,sports,mean context
models,,,,,,,
aim,0.703589,0.717085,0.681711,0.682661,0.693656,0.731142,0.701641
cas,0.698546,0.714773,0.672241,0.669232,0.682526,0.729582,0.694483
covsal,0.753049,0.763897,0.735464,0.737088,0.737128,0.770830,0.749576
gbvs,0.753201,0.764895,0.734245,0.736809,0.735714,0.784198,0.751510
icf,0.705782,0.718012,0.685956,0.681121,0.690002,0.721120,0.700332
ik,0.670892,0.686282,0.645943,0.659359,0.666581,0.692061,0.670186
sun,0.615479,0.619786,0.608496,0.564438,0.608479,0.602998,0.603279
mean model,0.700077,0.712104,0.680580,0.675816,0.687727,0.718847,0.695858


In [23]:
df_f07_mean_model = df_f07.loc['mean model']
df_f07_mean_context = df_f07.transpose().loc['mean context']

### Create Mean Matrices

In [24]:
mean_models = pd.DataFrame([df_mean_model, df_f03_mean_model, df_f05_mean_model, df_f07_mean_model], index=[0,0.3,0.5,0.7])

mean_models = mean_models.transpose()
mean_models = mean_models.loc[['all', 'person', 'no person', 'animal', 'electronic', 'sports']]
mean_models

,0.0,0.3,0.5,0.7
context,,,,
all,0.701773,0.697610,0.699069,0.700077
person,0.710203,0.708264,0.710575,0.712104
no person,0.688107,0.680321,0.680416,0.680580
animal,0.682807,0.674728,0.675339,0.675816
electronic,0.684005,0.683037,0.685660,0.687727
sports,0.724303,0.717384,0.718032,0.718847


In [25]:
mean_contexts = pd.DataFrame([df_mean_context, df_f03_mean_context, df_f05_mean_context, df_f07_mean_context], index=[0,0.3,0.5,0.7])

mean_contexts = mean_contexts.transpose()
mean_contexts = mean_contexts.loc[['aim', 'cas', 'covsal', 'gbvs', 'icf', 'ik', 'sun']]
mean_contexts

,0.0,0.3,0.5,0.7
aim,0.697163,0.699687,0.700793,0.701641
cas,0.689103,0.692001,0.693586,0.694483
covsal,0.743779,0.747156,0.748292,0.749576
gbvs,0.748130,0.750355,0.751127,0.751510
icf,0.696548,0.698487,0.699239,0.700332
ik,0.663201,0.667707,0.669076,0.670186
sun,0.592978,0.599509,0.601827,0.603279


## Diff of Context 

#### Drop DG for diff

In [26]:
df1 = df.loc[['aim','cas', 'covsal', 'gbvs', 'icf', 'ik', 'sun', 'mean model']]
df1

context,all,person,no person,animal,electronic,sports,mean context
models,,,,,,,
aim,0.698040,0.707347,0.682952,0.681585,0.683133,0.729921,0.697163
cas,0.692052,0.703851,0.672925,0.667086,0.670638,0.728063,0.689103
covsal,0.746774,0.754006,0.735052,0.733442,0.726257,0.767142,0.743779
gbvs,0.749465,0.758711,0.734478,0.734344,0.728549,0.783232,0.748130
icf,0.701568,0.711141,0.686049,0.679900,0.682979,0.717650,0.696548
ik,0.663008,0.673263,0.646383,0.656406,0.652015,0.688133,0.663201
sun,0.603264,0.599431,0.609479,0.560426,0.589477,0.595790,0.592978
mean model,0.701773,0.710203,0.688107,0.682807,0.684005,0.724303,0.698533


In [27]:
df_diff_f03 = ((df_f03 - df1) / df1)*100
df_diff_f05 = ((df_f05 - df1) / df1)*100
df_diff_f07 = ((df_f07 - df1) / df1)*100

#### Get mean of models and mean of contexts in diff out

#### Diff Models (analyse across contexts)

In [28]:
mean_model_diff_f03 = df_diff_f03.loc['mean model']
mean_model_diff_f05 = df_diff_f05.loc['mean model']
mean_model_diff_f07 = df_diff_f07.loc['mean model']

mean_models_diff = pd.DataFrame([mean_model_diff_f03, 
                            mean_model_diff_f05,
                            mean_model_diff_f07], index=[0.3,0.5,0.7])

mean_models_diff = mean_models_diff.transpose()
mean_models_diff

,0.3,0.5,0.7
context,,,
all,-0.593155,-0.385330,-0.241669
person,-0.272995,0.052389,0.267697
no person,-1.131463,-1.117680,-1.093892
animal,-1.183259,-1.093754,-1.023985
electronic,-0.141614,0.241954,0.544030
sports,-0.955325,-0.865868,-0.753258
mean context,-0.712316,-0.527479,-0.382908


In [29]:
mean_context_diff_f03 = df_diff_f03.transpose().loc['mean context']
mean_context_diff_f05 = df_diff_f05.transpose().loc['mean context']
mean_context_diff_f07 = df_diff_f07.transpose().loc['mean context']

mean_context_diff = pd.DataFrame([mean_context_diff_f03, 
                            mean_context_diff_f05,
                            mean_context_diff_f07], index=[0.3,0.5,0.7])

mean_context_diff = mean_context_diff.transpose()
mean_context_diff

,0.3,0.5,0.7
models,,,
aim,0.362052,0.520655,0.642290
cas,0.420669,0.650640,0.780828
covsal,0.454015,0.606769,0.779441
gbvs,0.297443,0.400656,0.451862
icf,0.278337,0.386326,0.543306
ik,0.679390,0.885814,1.053258
sun,1.101376,1.492293,1.737272
mean model,-0.712316,-0.527479,-0.382908


## Vizualize relative errors

## Diff of Models

## List all the df and save them

In [30]:
df_feaces = pd.DataFrame(data=faces)

In [31]:
os.chdir('../notebooks/data/')

In [32]:
df.to_csv('scores_' + plot_metric + '.csv')
df_feaces.to_csv('scores_faces_' + plot_metric + '.csv')
df_f03.to_csv('scores_f03_' + plot_metric + '.csv')
df_f05.to_csv('scores_f05_' + plot_metric + '.csv')
df_f07.to_csv('scores_f07_' + plot_metric + '.csv')

mean_models.to_csv('scores_mean_models_' + plot_metric + '.csv')
mean_contexts.to_csv('scores_mean_contexts_' + plot_metric + '.csv')

mean_models_diff.to_csv('scores_mean_models_diff_' + plot_metric + '.csv')
mean_context_diff.to_csv('scores_mean_contexts_diff_' + plot_metric + '.csv')